In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns
import numpy as np
%matplotlib inline
from sklearn import model_selection, preprocessing
from sklearn.linear_model import LinearRegression

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
print("train:", train.shape)
print("test:" ,test.shape)

In [ ]:
y = train.price_doc

In [ ]:
plt.plot(y)

In [ ]:
#histogramme de log de price
plt.hist(np.log(y), bins=100)
plt.title('Histogram of log-price')

In [ ]:
#spécifier les colones numériques et les colones string
dtypes = train.iloc[:,2:-1].dtypes
string_cols = dtypes[dtypes == object].index
num_cols = dtypes[dtypes != object].index

In [ ]:
#description des colones non numériques (les catégories)
def categorical_summary(data, col):
    filled_values = sum(data[col].notnull())
    missing = sum(data[col].isnull())
    cardinality = len(data[col].unique())
    print(col.upper())
    print('-------------------------------')
    print('filled count: %s' %filled_values)
    print('missing count: %s' %missing)
    print('cardinality: %s' %cardinality)
    print(data[col].value_counts())
    print('\n')

In [ ]:
for col in string_cols:
    categorical_summary(train, col)

In [ ]:
#transformation des yes no lists
yes_no_list = ['culture_objects_top_25', 'thermal_power_plant_raion', 'incineration_raion', 'oil_chemistry_raion',
          'radiation_raion', 'railroad_terminal_raion', 'big_market_raion', 'nuclear_reactor_raion',
          'detention_facility_raion', 'water_1line', 'big_road1_1line', 'railroad_1line']
for col in yes_no_list:
    train.ix[train[col]=='yes', col]=1
    train.ix[train[col]=='no', col]=0
    test.ix[test[col]=='yes', col]=1
    test.ix[test[col]=='no', col]=0

In [ ]:
#transformation de ecology
train.ix[train["ecology"]=='poor', "ecology"]=0
train.ix[train["ecology"]=='good', "ecology"]=2
train.ix[train["ecology"]=='excellent', "ecology"]=3
train.ix[train["ecology"]=='satisfactory', "ecology"]=1
test.ix[test["ecology"]=='poor', "ecology"]=0
test.ix[test["ecology"]=='good', "ecology"]=2
test.ix[test["ecology"]=='excellent', "ecology"]=3
test.ix[test["ecology"]=='satisfactory', "ecology"]=1

In [ ]:
#determiner le mean de ecology sans missing data
i = 0;
s = 0;
for lig in train["ecology"]:
    if lig != "no data" :
        i=i+1
        s=s+lig
mean = s/i   
print (mean)
i = 0;
s = 0;
for lig in test["ecology"]:
    if lig != "no data" :
        i=i+1
        s=s+lig
mean = s/i   
print (mean)

In [ ]:
#remplacer la missing data par le mean de ecology
train.ix[train["ecology"]=='no data', "ecology"]=1
test.ix[test["ecology"]=='no data', "ecology"]=1

In [ ]:
train.ix[train["product_type"]=='Investment', "product_type"]=1
train.ix[train["product_type"]=='OwnerOccupier', "product_type"]=0
test.ix[test["product_type"]=='Investment', "product_type"]=1
test.ix[test["product_type"]=='OwnerOccupier', "product_type"]=0
print (test.head)

In [ ]:
from sklearn.base import TransformerMixin
class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
        if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
        index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)
train = DataFrameImputer().fit_transform(train)
test = DataFrameImputer().fit_transform(test)

In [ ]:
y = train.price_doc
train_X = train.drop(["price_doc","id","timestamp","sub_area"],axis=1)
# instantiate
linreg = LinearRegression()

# fit the model to the training data (learn the coefficients)
linreg.fit(train_X, y)

In [ ]:
print(linreg.intercept_)
print(linreg.coef_)

In [ ]:
from sklearn import metrics
test_X = test.drop(["id","timestamp","sub_area"],axis=1)
y_pred = linreg.predict(test_X)